In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time


import scipy.interpolate
from scipy.ndimage import gaussian_filter
import tifffile
import seaborn as sns

from IPython.display import HTML
import matplotlib.animation as animation
from tqdm.notebook import tqdm, trange
from scipy.optimize import curve_fit
#import gaussian
from skimage.filters import gaussian
from scipy.interpolate import RectBivariateSpline
from PIL import Image
import itertools
from utils.graph_utils import image_to_int8

from scipy.ndimage import gaussian_gradient_magnitude
import ants

In [2]:
PROJECT_PATH = 'C:/Users/amityu/DataspellProjects/gel_sheets/'
DATA_PATH = 'C:/Users/amityu/Gel_Sheet_Data/'
#movie = 'Control'
#movie = '130721'
#movie ='140721'
#movie ='150721'
#movie ='100621'
movie ='280523 AM100 568'
ADD_PATH = os.path.join(PROJECT_PATH, "add_data/")


MOVIE_PATH = DATA_PATH +  movie + '/'
GRAPH_PATH = 'C:/Users/amityu/Gel_Sheet_Graph/'

In [3]:

gel = np.load(MOVIE_PATH + 'np/gel.npy')
gel[:,61:,:,:] = 0
gel = np.transpose(gel, (3,2,1,0))

In [6]:
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

t =0; z1 = 0; z2 = 15; y1 = 0; y2 = 511; x1 = 0; x2 = 511
mask = np.zeros_like(gel[:,:,:,0])
mask[x1:x2,y1:y2,z1:z2] = 1
mask = ants.from_numpy(mask)
moving_mask = np.zeros_like(gel[:,:,:,0])
moving_mask[x1:x2,y1:y2,0:z1 ] = 1
moving_mask = ants.from_numpy(moving_mask)
gel_ant = ants.from_numpy(gel)
fixed_image = ants.from_numpy(gel[:,:,:,0])
mytx = ants.motion_correction(gel_ant,  fixed=fixed_image, type_of_transform='Rigid', mask=mask)
images = mytx['motion_corrected'].numpy()

#new_image = ants.registration(fixed_image, moving=ants.from_numpy(gel[:,:,:,12]), type_of_transform='Rigid')['warpedmovout']
for i in range(gel.shape[3]):
    tifffile.imsave(MOVIE_PATH + 'np/motion_Z15_' + str(i+1) + '.tif', images[:,:,:,i])
#tifffile.imsave(MOVIE_PATH + 'np/image12from0nomask.tif', new_image.numpy())
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))


2023-09-05 16:50:38
2023-09-05 16:55:52


In [5]:
np.save(MOVIE_PATH + 'np/motion_corrected.npy', images)

In [7]:
tifffile.imsave(MOVIE_PATH + 'np/slice.tif', images[200,:,:,:])

In [37]:
tifffile.imsave(MOVIE_PATH + 'np/original_image0.tif', gel[:,:,:,0])
tifffile.imsave(MOVIE_PATH + 'np/original_image12.tif', gel[:,:,:,12])


In [17]:
 for i in range(gel.shape[3]):
    t = ants.read_transform(mytx['motion_parameters'][i][0])
    ants.write_transform(t, PROJECT_PATH + 'add_data/280523 AM100 568/transform/transform' + str(i+1) + '.mat')


RuntimeError: D:\a\ANTsPy\ANTsPy\itksource\Modules\IO\TransformBase\src\itkTransformFileWriterSpecializations.cxx:159:
ITK ERROR: TransformFileWriterTemplate(000002639C8F5F10): Could not create Transform IO object for writing file C:/Users/amityu/DataspellProjects/gel_sheets/add_data/280523 AM100 568/transform/transform1.MAT
  Tried to create one of the following:
    HDF5TransformIOTemplate
    HDF5TransformIOTemplate
    MatlabTransformIOTemplate
    MatlabTransformIOTemplate
    TxtTransformIOTemplate
    TxtTransformIOTemplate
  You probably failed to set a file suffix, or
    set the suffix to an unsupported type.


In [18]:
t = ants.read_transform(PROJECT_PATH + 'add_data/280523 AM100 568/transform/transform' + str(5+1) + '.mat')
